# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd './drive/MyDrive/hasoc/task1'

/content/drive/MyDrive/hasoc/task1


# Installing Libraries

In [3]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install googletrans==4.0.0-rc1
# !pip install googletrans
!pip install accelerate -U
!pip install evaluate

# Imports and Set Seed

In [4]:
import random
import torch
import numpy as np

GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


# Reading Translated Data

In [5]:
# unique_labels = df['label'].unique()
# label2id = {label: id for id, label in enumerate(unique_labels)}
# id2label = {id: label for id, label in enumerate(unique_labels)}
# num_labels = len(unique_labels)
unique_labels = ['HOF', 'NOT']
label2id = {'HOF': 0, 'NOT': 1}
id2label = {0: 'HOF', 1: 'NOT'}
num_labels = 2

In [6]:
id2label

{0: 'HOF', 1: 'NOT'}

# Defining Model and Tokenize Dataset

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_checkpoint = "keshan/SinhalaBERTo"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at keshan/SinhalaBERTo and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [9]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset, DatasetDict
from datasets import Dataset
from datasets import load_dataset, concatenate_datasets

sold_train = load_dataset('sinhala-nlp/SOLD', split='train')
sold_test = load_dataset('sinhala-nlp/SOLD', split='test')

# sold_dataset = sold_train.concat(sold_test)
# sold_dataset = concatenate_datasets([sold_train, sold_test])

# val_df = pd.read_csv('sinhala_val_dataset.csv', index_col = 4)


# Convert the training and testing sets back to datasets
# train_dataset = Dataset.from_pandas(train_df)
# train_dataset = sold_dataset
train_dataset = sold_train
val_dataset = sold_test

old_feature_name = "label"
new_feature_name = "hate_label"
train_dataset = train_dataset.rename_column(old_feature_name, new_feature_name)
val_dataset = val_dataset.rename_column(old_feature_name, new_feature_name)

def map_hate_label_to_label(example):
    if example['hate_label'] == 'OFF':
        example['label'] = 0
    elif example['hate_label'] == 'NOT':
        example['label'] = 1
    return example

train_dataset = train_dataset.map(map_hate_label_to_label)
val_dataset = val_dataset.map(map_hate_label_to_label)

# val_dataset = Dataset.from_pandas(val_df)



# Tokenization
# tokenizer = AutoTokenizer.from_pretrained("your_pretrained_model")  # Replace with your tokenizer name
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_val_datasets = val_dataset.map(tokenize_function, batched=True)


tokenized_datasets = DatasetDict({"train": tokenized_train_datasets, "validation": tokenized_val_datasets})

Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
train_dataset

Dataset({
    features: ['post_id', 'text', 'tokens', 'rationales', 'hate_label', 'label'],
    num_rows: 7500
})

In [11]:
val_dataset

Dataset({
    features: ['post_id', 'text', 'tokens', 'rationales', 'hate_label', 'label'],
    num_rows: 2500
})

In [12]:
tokenized_datasets
# tokenized_train_datasets

DatasetDict({
    train: Dataset({
        features: ['post_id', 'text', 'tokens', 'rationales', 'hate_label', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7500
    })
    validation: Dataset({
        features: ['post_id', 'text', 'tokens', 'rationales', 'hate_label', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2500
    })
})

In [13]:
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer,
#     mlm=False  # Set to True if your task is Masked Language Modeling
# )
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Train Model

In [14]:
training_args = TrainingArguments(
    output_dir="./output",
    evaluation_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    learning_rate=2e-5,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=processed_dataset["train"],
    # eval_dataset=processed_dataset["validation"],
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator
)

# trainer.train(resume_from_checkpoint="last-checkpoint")
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.552700,0.416851
2,0.359100,0.447092


TrainOutput(global_step=1876, training_loss=0.435424495607551, metrics={'train_runtime': 312.7639, 'train_samples_per_second': 47.959, 'train_steps_per_second': 5.998, 'total_flos': 638576688547632.0, 'train_loss': 0.435424495607551, 'epoch': 2.0})

In [16]:
predictions = trainer.predict(tokenized_datasets["validation"])
# print(predictions.predictions.shape, predictions.label_ids.shape)
preds = np.argmax(predictions.predictions, axis=-1)

In [20]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Evaluate on validation data
results = trainer.evaluate()

# Calculate metrics
validation_loss = results["eval_loss"]
accuracy = accuracy_score(tokenized_datasets["validation"]["label"], preds)
precision, recall, f1, _ = precision_recall_fscore_support(tokenized_datasets["validation"]["label"], preds, average="macro")

# Print the metrics
print(f"Validation Loss: {validation_loss}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Validation Loss: 0.41685059666633606
Accuracy: 0.8228
Precision: 0.821552855703776
Recall: 0.8074986316365627
F1 Score: 0.8126092724823611


In [18]:
test_df = pd.read_csv('sinhala_test_translated.csv', index_col = 0)


from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset, DatasetDict

# Convert the training and testing sets back to datasets
test_dataset = Dataset.from_pandas(test_df)


tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)


tokenized_datasets["test"] = tokenized_test_datasets

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [19]:
# Save model predictions on the test dataset
test_predictions = trainer.predict(tokenized_datasets["test"])
test_preds = np.argmax(test_predictions.predictions, axis=-1)

# Save the predictions to a file (e.g., CSV)
# test_predictions_df = pd.DataFrame({"predictions": test_preds})
test_predictions_df = test_df.copy()
test_predictions_df['predictions'] = test_preds
test_predictions_df.to_csv("test_predictions_sinhalaRoberta.csv")